In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install line-bot-sdk flask flask-ngrok

     |████████████████████████████████| 68 kB 4.0 MB/s 


In [3]:
# 引用flask套件
from flask import Flask, request, abort, jsonify, render_template

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 引用json套件
import json
# 引用line套件
from linebot import (
    LineBotApi, WebhookHandler
)
# 引用驗證消息用的套件
from linebot.exceptions import (
    InvalidSignatureError
)
# 引用line的消息套件
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, FollowEvent, ImageMessage, ImageSendMessage, RichMenu, LocationSendMessage,
    TemplateSendMessage, PostbackEvent, FlexSendMessage, VideoSendMessage, StickerSendMessage, AudioSendMessage, MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,CameraAction, CameraRollAction, LocationAction, QuickReply, QuickReplyButton
)

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction,
    QuickReply, QuickReplyButton
)

# 引入按鍵模板

from linebot.models.template import (
    ButtonsTemplate, CarouselTemplate, ConfirmTemplate, ImageCarouselTemplate

)


# 圖片下載與上傳專用
import urllib.request
import os

# 其他用到的套件
import random
from datetime import date

In [4]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("Channel access token")
handler = WebhookHandler("Channel secret")

In [5]:
# 啟動server對外接口，使Line能丟消息進來
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [6]:
def detect_json_array_to_new_message_array(fileName):
    # 開啟檔案，轉成json
    with open(fileName, encoding='utf8') as f:
        jsonArray = json.load(f)

    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')

        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'video':
            returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))

            # 回傳
    return returnArray

In [7]:
'''
菜單設定檔

    設定圖面大小、按鍵名與功能
    
'''

menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "圖文選單1",
  "chatBarText": "看更多",
  "areas": [
    {
      "bounds": {
        "x": 0,
        "y": 260,
        "width": 830,
        "height": 721
      },
      "action": {
        "type": "postback",
        "text": "蔬果價格排行表",
        "data": "price_level"
      }
    },
    {
      "bounds": {
        "x": 830,
        "y": 258,
        "width": 845,
        "height": 722
      },
      "action": {
        "type": "message",
        "text": "看水果"
      }
    },
    {
      "bounds": {
        "x": 1684,
        "y": 259,
        "width": 812,
        "height": 721
      },
      "action": {
        "type": "message",
        "text": "看蔬菜"
      }
    },
    {
      "bounds": {
        "x": 0,
        "y": 972,
        "width": 825,
        "height": 710
      },
      "action": {
        "type": "message",
        "text": "蔬果報報"
      }
    },
    {
      "bounds": {
        "x": 826,
        "y": 976,
        "width": 848,
        "height": 710
      },
      "action": {
        "type": "postback",
        "text": "推薦購買組合",
        "data": "recommend_buy_set"
      }
    },
    {
      "bounds": {
        "x": 1670,
        "y": 976,
        "width": 830,
        "height": 710
      },
      "action": {
        "type": "message",
        "text": "分享"
      }
    },
    {
      "bounds": {
        "x": 5,
        "y": 0,
        "width": 1825,
        "height": 259
      },
      "action": {
        "type": "message",
        "text": "選地區"
      }
    },
    {
      "bounds": {
        "x": 1835,
        "y": 0,
        "width": 655,
        "height": 245
      },
      "action": {
        "type": "message",
        "text": "意見反應"
      }
    }
  ]
}
"""

In [8]:
# 創建QuickReplyButton
## 點擊後，以Postback事件回應Server 
postbackQRB_0 = QuickReplyButton(
    action=PostbackAction(label="200以內", data="set_0")
)

postbackQRB_1 = QuickReplyButton(
    action=PostbackAction(label="200-500", data="set_1")
)

postbackQRB_2 = QuickReplyButton(
    action=PostbackAction(label="500-1000", data="set_2")
)

postbackQRB_3 = QuickReplyButton(
    action=PostbackAction(label="1000以上", data="set_3")
)

## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [postbackQRB_0, postbackQRB_1, postbackQRB_2, postbackQRB_3]
)

# 製作TextSendMessage，並將剛封裝的QuickReply放入
quick_reply_text_send_message = TextSendMessage(text='請選擇您這週的買菜預算，我們會推薦最划算的組合給您參考', quick_reply=quickReplyList)

In [9]:
'''

載入前面的圖文選單設定，

並要求line_bot_api將圖文選單上傳至Line
    

'''

from linebot.models import RichMenu
import requests

menuJson=json.loads(menuRawData)

lineRichMenuId_one = line_bot_api.create_rich_menu(rich_menu=RichMenu.new_from_json_dict(menuJson))
print(lineRichMenuId_one)

richmenu-dd935b50da43ded775ac047858cd4efb


In [10]:
'''

將先前準備的菜單照片，以Post消息寄發給Line

    載入照片
    要求line_bot_api，將圖片傳到先前的圖文選單id


'''


uploadImageFile=open("/content/drive/MyDrive/AI課程檔案/專題相關/line_chatbot/圖四.001.jpg",'rb')

setImageResponse = line_bot_api.set_rich_menu_image(lineRichMenuId_one,'image/jpeg',uploadImageFile)

print(setImageResponse)

None


In [11]:
'''

1. 取得用戶個資，並存回伺服器
2. 回應用戶，歡迎用的文字消息

'''
# 這部分是光超要寫的

# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

from linebot.models import(
    TextSendMessage, ImageSendMessage
)

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def handle_follow_event(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)

    # 當用戶關注時，回覆以下訊息
    text_1 = "個資已取"
    line_bot_api.reply_message(event.reply_token, [TextSendMessage(text=text_1)])
    # 當用戶關注時，綁定圖文選單
    line_bot_api.link_rich_menu_to_user(event.source.user_id, lineRichMenuId_one)

尚未確認：
1. 到資料庫查詢使用者地區

In [12]:
# 當收到Poskback的處理方法
@handler.add(PostbackEvent)
def handle_post_message(event):
    if event.postback.data.startswith('price_level'):
      # 推播蔬果價格排行
      # 根據使用者id 抓取使用者是哪一區(需要和資料庫記錄使用者資訊部分綁定，尚未處理)
      user_id = event.source.user_id
      file_path = '/content/drive/MyDrive/AI課程檔案/專題相關/line_chatbot/price_rank_北區.json'
      # 根據使用者地區 抓取該區的price_rank.json檔(上面尚未處理，目前先假設以北區為例)
      FlexMessage = json.load(open(file_path,'r',encoding='utf-8'))
      line_bot_api.reply_message(event.reply_token, FlexSendMessage('每日價格排行榜',FlexMessage))
        
    elif event.postback.data.startswith('recommend_buy_set'):
      # 跳出氣泡視窗請使用者選擇預算
      line_bot_api.reply_message(event.reply_token, quick_reply_text_send_message)

    elif event.postback.data.startswith('set'):
      # 使用者選擇完預算後，依照對應預算推薦購物組合
      # 先根據postback的data標籤，判斷使用者是哪一個預算
      num = event.postback.data[-1]
      file_path = f'/content/drive/MyDrive/AI課程檔案/專題相關/line_chatbot/recommend_buy_{num}.json'
      FlexMessage = json.load(open( file_path,'r',encoding='utf-8'))
      line_bot_api.reply_message(event.reply_token, FlexSendMessage('最划算的購物組合',FlexMessage))
      

In [13]:
'''

檢視帳號內，有哪些選單
    要求line_bot_api，向line查詢我方的圖文選單列表
    打印

'''

rich_menu_list = line_bot_api.get_rich_menu_list()


for rich_menu in rich_menu_list:
  print(rich_menu.rich_menu_id)


richmenu-c726695cd5da6ac1b5b1ec846418f1df
richmenu-333cb9678d25394da7bb103b76a598a4
richmenu-c59b7772af6efbe74b76a4488132c569
richmenu-962b8a9b2c4825d6f5b9e05a32449020
richmenu-dd935b50da43ded775ac047858cd4efb
richmenu-6d5bec2b3b71169166ae55ac4441f458
richmenu-cec24e437d9b6d32f789d1439fc7e5a4
richmenu-b4eb4d30a1f518f16c346efac7a0acd3
richmenu-bbaf34c119b0f71979081e84cb3fd675
richmenu-7da79a1d79c1a519538e32ffafffe51b
richmenu-d44eea927b3b6c2a02020c8c78f34bac
richmenu-accbdeec3692522727c19cf750467450
richmenu-7edff905e1e99929fc2ed7cd2a39c1ca
richmenu-6a06960dfc8ff67de46521bfb60bb863
richmenu-a984d91e4825447254c19ac7d592a762


In [14]:
# 運行主程序
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://609db899628b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Jul/2021 15:53:51] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2021 15:53:51] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2021 15:53:52] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2021 15:54:11] "POST /callback HTTP/1.1" 200 -


{"destination":"U9719c8a41962aba3fa48bf87294ecfb9","events":[{"type":"unfollow","timestamp":1626969250543,"source":{"type":"user","userId":"U02f652445e532f1ce106e8f3332018af"},"mode":"active"}]}
{"destination":"U9719c8a41962aba3fa48bf87294ecfb9","events":[{"type":"follow","timestamp":1626969252406,"source":{"type":"user","userId":"U02f652445e532f1ce106e8f3332018af"},"replyToken":"217feb3e6b344c47b45302ff5379778b","mode":"active"}]}


...

127.0.0.1 - - [22/Jul/2021 15:55:41] "POST /callback HTTP/1.1" 200 -


{"destination":"U9719c8a41962aba3fa48bf87294ecfb9","events":[{"type":"postback","postback":{"data":"recommend_buy_set"},"timestamp":1626969344021,"source":{"type":"user","userId":"U02f652445e532f1ce106e8f3332018af"},"replyToken":"8d736637a90441de87af7b9d7e0c2c37","mode":"active"}]}


127.0.0.1 - - [22/Jul/2021 15:55:45] "POST /callback HTTP/1.1" 200 -


{"destination":"U9719c8a41962aba3fa48bf87294ecfb9","events":[{"type":"postback","postback":{"data":"set_1"},"timestamp":1626969346038,"source":{"type":"user","userId":"U02f652445e532f1ce106e8f3332018af"},"replyToken":"a3bdfc617c614584a8b4e50003cc9434","mode":"active"}]}


127.0.0.1 - - [22/Jul/2021 15:55:47] "POST /callback HTTP/1.1" 200 -
